references <br>
http://blog.theeluwin.kr/post/147587579528/python-crfsuite%EB%A5%BC-%EC%82%AC%EC%9A%A9%ED%95%B4%EC%84%9C-%ED%95%9C%EA%B5%AD%EC%96%B4-%EC%9E%90%EB%8F%99-%EB%9D%84%EC%96%B4%EC%93%B0%EA%B8%B0%EB%A5%BC-%ED%95%99%EC%8A%B5%ED%95%B4%EB%B3%B4%EC%9E%90

In [1]:
import os
import glob
import mxnet as mx
from mxnet.gluon import nn, rnn
import gluonnlp as nlp
import pandas as pd
import numpy as np
from mxnet import gluon, autograd
from mxnet import nd
import mxnet as mx
import time
import itertools
from tqdm import tqdm
import multiprocessing as mp
from konlpy.tag import Twitter
twitter = Twitter()

/home/chatbot/anaconda3/envs/mxnet/lib/python3.6/site-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


## read all review files

In [2]:
path = "101 movie/reviews_2012"
allFiles = glob.glob(path + "/*.csv")
list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_,index_col=None, header=0, sep='\t')
    list_.append(df)
reviews = pd.concat(list_, axis = 0, ignore_index = True)

In [3]:
reviews[:10]

,Unnamed: 0,comments,reg_date,source,pn,topic,mid
0,0,BEST가슴세개녀가 너무 인상깊었다,2013.02.18,n,1,3,20125647
1,1,"BEST결국 리콜사에서 환상을 끝내주게 경험했다는 얘기같네요, 리메이크 수준도 괜찮...",2014.03.01,n,1,3,20125647
2,2,BEST결국 가상이였을까 현실이였을까..많은 생각을 하게만든 영화,2013.09.28,n,1,0,20125647
3,3,"BEST원작의 색안경을 빼고 보면 참 잘만든 영화, 원작팬들을 위한 오마쥬에도 충실함",2013.10.09,n,1,0,20125647
4,4,BEST볼만하네요... 적이 좀 허무하게 무너지는 경향이 있지만 나름 시간때우기용...,2013.06.13,n,1,0,20125647
5,5,여배우가 이쁜데 영화가 너무 길어,2013.07.09,n,0,1,20125647
6,6,"스토리, 액션, 멜로 , 반전... 전부 탄탄한 스케일의 영화.",2014.05.05,n,0,0,20125647
7,7,생각보다 잘만들어진 리메이크 작이었음. 스토리도 괜찮았고 나름 볼거리도 있어서 만...,2014.05.17,n,1,0,20125647
8,8,스토리가 괜춘했음!! 생각보다 시간도 잘 갔어요,2013.08.18,n,0,0,20125647
9,9,볼만한데..너무 까는 사람이 많네 ㅋㅋ,2014.05.06,n,1,3,20125647


In [3]:
len(reviews)

674949

## save all files to one

In [4]:
# export all files
reviews.comments.to_csv("all_reviews.txt", header=None, index=None, sep=" ", mode="a")

# export a part of all files for sample
reviews.comments[:100].to_csv("samples.txt", header=None, index=None, sep=" ", mode="a")

comments = reviews.comments[:100]
comments[:10]

0                                BEST가슴세개녀가 너무 인상깊었다  
1    BEST결국 리콜사에서 환상을 끝내주게 경험했다는 얘기같네요, 리메이크 수준도 괜찮...
2               BEST결국 가상이였을까 현실이였을까..많은 생각을 하게만든 영화  
3    BEST원작의 색안경을 빼고 보면 참 잘만든 영화, 원작팬들을 위한 오마쥬에도 충실함  
4    BEST볼만하네요...  적이 좀 허무하게 무너지는 경향이 있지만 나름 시간때우기용...
5                                 여배우가 이쁜데 영화가 너무 길어  
6                스토리, 액션, 멜로 , 반전... 전부 탄탄한 스케일의 영화.  
7    생각보다 잘만들어진 리메이크 작이었음.  스토리도 괜찮았고 나름 볼거리도 있어서 만...
8                         스토리가 괜춘했음!! 생각보다 시간도 잘 갔어요  
9                              볼만한데..너무 까는 사람이 많네 ㅋㅋ  
Name: comments, dtype: object

## get all actors

In [11]:
all_actors = pd.read_csv("101 movie/all_actors.csv",index_col=None, header=None, sep=',')
all_actors[:10]
len(all_actors)

1423

## get all directors

In [14]:
all_directors = pd.read_csv("101 movie/all_directors.csv",index_col=None, header=None, sep=',')
all_directors[:10]
len(all_directors)

519

In [4]:
people = pd.read_csv("101 movie/all_people.csv",index_col=None, header=None, sep=',')
people
len(people)

1941

## people.csv에 있는 사람이름 raw data에 태깅하기

In [9]:
# -*- coding: utf-8 -*-
from __future__ import unicode_literals, print_function

import re
import codecs

def get_sentence(raw_sentences, tag):
    for sentence in raw_sentences:
        text = re.sub(r'(\ )+', ' ', sentence).strip()
        text = text.replace('"', '')
        word_list = twitter.pos(text)
        taggeds = []
        for word in word_list:
            hit = False
            if (word[1] == "Noun") and (len(word[0]) > 1):
                for person in people[1]:
                    name = person.split(" ")
                    for aName in name:
                        if (word[0] == aName):
                            hit = True
                            break
                if (hit):
                    taggeds.append(('{}/'+tag).format(word[0]))
                else:
                    taggeds.append('{}/O'.format(word[0]))
            else:
                taggeds.append('{}/O'.format(word[0]))
        yield taggeds

# all_reviews, all_people
def raw2corpus_(raw_path, train_path, tag, corpus_path):
    raw = codecs.open(raw_path, encoding='utf-8')
    raw_sentences = raw.read().split('\n')
    corpus = codecs.open(corpus_path, 'w', encoding='utf-8')
    
    train_data = pd.read_csv(train_path,index_col=None, header=None, sep=',')
    sentences = []
    for taggeds in get_sentence(raw_sentences, tag):
        sentences.append(' '.join(taggeds))
    corpus.write('\n'.join(sentences))
    #return sentences

In [108]:
"""
def raw2corpus_(raw_path, train_path, tag, corpus_path):
    raw = codecs.open(raw_path, encoding='utf-8')
    raw_sentences = raw.read().split('\n')
    corpus = codecs.open(corpus_path, 'w', encoding='utf-8')
    train_data = pd.read_csv(train_path,index_col=None, header=None, sep=',')
    sentences = []
    for raw_sentence in raw_sentences:
        if not raw_sentence:
            continue
        text = re.sub(r'(\ )+', ' ', raw_sentence).strip()
        text = text.replace('"', '')
        word_list = twitter.pos(text)
        taggeds = []
        for word in word_list:
            hit = False
            if (word[1] == "Noun") and (len(word[0]) > 1):
                for person in people[1]:
                    name = []
                    name.append(person)
                    split_name = person.split(" ")
                    name = name + split_name
                    for aName in name:
                        if (word[0] == aName):
                            hit = True
                            break
                if (hit):
                    taggeds.append(('{}/'+tag).format(word[0]))
                else:
                    taggeds.append('{}/O'.format(word[0]))
            else:
                taggeds.append('{}/O'.format(word[0]))
        sentences.append(' '.join(taggeds))
    corpus.write('\n'.join(sentences))
    return sentences
"""

In [10]:
#raw2corpus_("samples.txt", "101 movie/all_people.csv", "PERSON", "people_list.txt")
raw2corpus_("all_reviews.txt", "101 movie/all_people.csv", "PERSON", "data/all_taged_reviews.txt")

In [11]:
# all review data // all actor data // file for save
'''
raw2corpus_("samples.txt", "101 movie/all_actors.csv", "ACTOR", "actor_list.txt")
raw2corpus_("samples.txt", "101 movie/all_directors.csv", "DIRECTOR", "director_list.txt")
'''

'\nraw2corpus_("samples.txt", "101 movie/all_actors.csv", "ACTOR", "actor_list.txt")\nraw2corpus_("samples.txt", "101 movie/all_directors.csv", "DIRECTOR", "director_list.txt")\n'

In [3]:
# -*- coding: utf-8 -*-
from __future__ import unicode_literals, print_function

import re
import codecs
'''
def raw2corpus(raw_path, corpus_path):
    raw = codecs.open(raw_path, encoding='utf-8')
    raw_sentences = raw.read().split('\n')
    corpus = codecs.open(corpus_path, 'w', encoding='utf-8')
    sentences = []
    for raw_sentence in raw_sentences:
        if not raw_sentence:
            continue
        text = re.sub(r'(\ )+', ' ', raw_sentence).strip()
        text = text.replace('"', '')
        taggeds = []
        for i in range(len(text)):
            if i == 0:
                taggeds.append('{}/B'.format(text[i]))
            elif text[i] != ' ':
                successor = text[i - 1]
                if successor == ' ':
                    taggeds.append('{}/B'.format(text[i]))
                else:
                    taggeds.append('{}/I'.format(text[i]))
        sentences.append(' '.join(taggeds))
    corpus.write('\n'.join(sentences))
    return sentences
'''

'\ndef raw2corpus(raw_path, corpus_path):\n    raw = codecs.open(raw_path, encoding=\'utf-8\')\n    raw_sentences = raw.read().split(\'\n\')\n    corpus = codecs.open(corpus_path, \'w\', encoding=\'utf-8\')\n    sentences = []\n    for raw_sentence in raw_sentences:\n        if not raw_sentence:\n            continue\n        text = re.sub(r\'(\\ )+\', \' \', raw_sentence).strip()\n        text = text.replace(\'"\', \'\')\n        taggeds = []\n        for i in range(len(text)):\n            if i == 0:\n                taggeds.append(\'{}/B\'.format(text[i]))\n            elif text[i] != \' \':\n                successor = text[i - 1]\n                if successor == \' \':\n                    taggeds.append(\'{}/B\'.format(text[i]))\n                else:\n                    taggeds.append(\'{}/I\'.format(text[i]))\n        sentences.append(\' \'.join(taggeds))\n    corpus.write(\'\n\'.join(sentences))\n    return sentences\n'

In [9]:
#raw2corpus("all_reviews.txt", "train.txt")

['B/B E/I S/I T/I 속/I 옷/I 만/I 입/B 은/I 채/B 피/B 범/I 벅/I 이/I 된/B 김/B 민/I 희/I 표/B 정/I 소/B 름/I 쫙/B',
 'B/B E/I S/I T/I 살/I 인/I 은/I 부/B 당/I 했/I 지/I 만/I ./B ./I 여/B 자/I 의/I 인/B 생/I 은/I 참/B 슬/B 프/I 네/I 요/I 기/B 구/I 하/I 고/I ./B ./I ./I 남/B 자/I 를/I 위/I 해/I 마/B 지/I 막/I 에/I 사/B 랑/I 을/I 하/B 지/I 않/I 았/I 다/I 고/I 한/B 거/I 같/I 아/I 요/I ./B ./I ./I 사/B 실/I 은/I 누/B 구/I 보/I 다/I 행/B 복/I 해/I 지/I 고/I 싶/B 으/I 면/I 서/I ./B ./I ./I',
 'B/B E/I S/I T/I 이/I 선/I 균/I 은/I 대/B 사/I 가/I 잘/B 안/B 들/I 릴/I 정/I 도/I 로/I 가/B 끔/I 너/B 무/I 작/B 게/I 말/B 한/I 다/I ./I',
 'B/B E/I S/I T/I 간/I 만/I 에/I 정/B 말/I 빠/B 져/I 들/I 고/I 스/B 릴/I 러/I 다/B 운/I 영/B 화/I 였/I 다/I ./I 추/I 격/I 자/I 이/B 후/I 이/B 렇/I 게/I 제/B 대/I 로/I 된/I 스/B 릴/I 러/I 를/I 보/B 다/I 니/I !/I 평/I 점/I 들/I 이/I 너/B 무/I 박/B 하/I 다/I ./I 그/I 리/I 고/I 김/B 민/I 희/I 가/I 이/B 렇/I 게/I 연/B 기/I 를/I 잘/B 했/B 었/I 나/I ./I 캐/I 릭/I 터/I 에/I 너/B 무/I 나/I 도/I 잘/B 소/B 화/I 했/I 다/I ./I',
 'B/B E/I S/I T/I 마/I 음/I 이/I 아/B 픈/I 스/B 릴/I 러/I ./I ./I 화/B 차/I 에/I 몸/B 을/I 싣/B 기/I 로/I 한/B 그/B 녀/I 의/I 선/B 택/I 이/I 너/B 무/I 아/B 프/

In [16]:
def corpus2sent_gen(raws):
    for raw in raws:
        tokens = raw.split(' ')
        sentence = []
        for token in tokens:
            try:
                word, tag = token.split('/')
                if word and tag:
                    sentence.append([word, tag])
            except:
                pass
        yield sentence

In [12]:
corpus = codecs.open('data/all_taged_reviews.txt', encoding='utf-8').read()
raws = corpus.split('\n')
print(len(reviews), len(raws))

674949 2025100


In [ ]:
sentences = []
for sentence in corpus2sent_gen(raws):
    sentences.append(sentence)

In [7]:
def corpus2sent(path):
    corpus = codecs.open(path, encoding='utf-8').read()
    raws = corpus.split('\n')
    sentences = []
    for raw in raws:
        tokens = raw.split(' ')
        sentence = []
        for token in tokens:
            try:
                word, tag = token.split('/')
                if word and tag:
                    sentence.append([word, tag])
            except:
                pass
        sentences.append(sentence)
    return sentences

In [ ]:
sample = corpus2sent('people_list.txt')[0]
sample

In [15]:
#sample = corpus2sent('train.txt')[7]

In [44]:
'{}{}:word={}'.format("+", 0, "best")

'+0:word=best'

In [13]:
def index2feature(sent, i, offset):
    word, tag = sent[i + offset]
    if offset < 0:
        sign = ''
    else:
        sign = '+'
    return '{}{}:word={}'.format(sign, offset, word)

def word2features(sent, i):
    L = len(sent)
    word, tag = sent[i]
    features = ['bias']
    features.append(index2feature(sent, i, 0))
    if i > 1:
        features.append(index2feature(sent, i, -2))
    if i > 0:
        features.append(index2feature(sent, i, -1))
    else:
        features.append('bos')
    if i < L - 2:
        features.append(index2feature(sent, i, 2))
    if i < L - 1:
        features.append(index2feature(sent, i, 1))
    else:
        features.append('eos')
    return features

def sent2words(sent):
    return [word for word, tag in sent]

def sent2tags(sent):
    return [tag for word, tag in sent]

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

In [14]:
corpus = codecs.open('data/all_taged_reviews.txt', encoding='utf-8').read()
raws = corpus.split('\n')

In [ ]:
train_x = []
train_y = []
for sent in corpus2sent_gen(raws):
    train_x.append(sent2features(sent))
    train_y.append(sent2tags(sent))

In [ ]:
import pycrfsuite

train_sents = corpus2sent('data/all_taged_reviews.txt')
#test_sents = corpus2sent('test.txt')
train_x = [sent2features(sent) for sent in train_sents]
train_y = [sent2tags(sent) for sent in train_sents]
#test_x = [sent2features(sent) for sent in test_sents]
#test_y = [sent2tags(sent) for sent in test_sents]

In [23]:
train_sents

[[['BEST', 'O'],
  ['속옷', 'O'],
  ['만', 'O'],
  ['입은', 'O'],
  ['채', 'O'],
  ['피', 'O'],
  ['범벅', 'O'],
  ['이', 'O'],
  ['된', 'O'],
  ['김민희', 'PERSON'],
  ['표정', 'O'],
  ['소름', 'O'],
  ['쫙', 'O']],
 [['BEST', 'O'],
  ['살인', 'O'],
  ['은', 'O'],
  ['부당했지만', 'O'],
  ['..', 'O'],
  ['여자', 'O'],
  ['의', 'O'],
  ['인생', 'O'],
  ['은', 'O'],
  ['참', 'O'],
  ['슬프네요', 'O'],
  ['기구', 'O'],
  ['하고', 'O'],
  ['...', 'O'],
  ['남자', 'O'],
  ['를', 'O'],
  ['위해', 'O'],
  ['마지막', 'O'],
  ['에', 'O'],
  ['사랑', 'O'],
  ['을', 'O'],
  ['하지', 'O'],
  ['않았다고', 'O'],
  ['한', 'O'],
  ['거', 'O'],
  ['같아요', 'O'],
  ['...', 'O'],
  ['사실', 'O'],
  ['은', 'O'],
  ['누구', 'O'],
  ['보다', 'O'],
  ['행복해지고', 'O'],
  ['싶으면서', 'O'],
  ['...', 'O']],
 [['BEST', 'O'],
  ['이선균', 'PERSON'],
  ['은', 'O'],
  ['대사', 'O'],
  ['가', 'O'],
  ['잘', 'O'],
  ['안', 'O'],
  ['들릴', 'O'],
  ['정도', 'O'],
  ['로', 'O'],
  ['가끔', 'O'],
  ['너무', 'O'],
  ['작게', 'O'],
  ['말', 'O'],
  ['한다', 'O'],
  ['.', 'O']],
 [['BEST', 'O'],
  ['간만', 'O'],
  ['에', 

In [ ]:
test_sents

In [24]:
train_x

[[['bias', '+0:word=BEST', 'bos', '+2:word=만', '+1:word=속옷'],
  ['bias', '+0:word=속옷', '-1:word=BEST', '+2:word=입은', '+1:word=만'],
  ['bias',
   '+0:word=만',
   '-2:word=BEST',
   '-1:word=속옷',
   '+2:word=채',
   '+1:word=입은'],
  ['bias', '+0:word=입은', '-2:word=속옷', '-1:word=만', '+2:word=피', '+1:word=채'],
  ['bias', '+0:word=채', '-2:word=만', '-1:word=입은', '+2:word=범벅', '+1:word=피'],
  ['bias', '+0:word=피', '-2:word=입은', '-1:word=채', '+2:word=이', '+1:word=범벅'],
  ['bias', '+0:word=범벅', '-2:word=채', '-1:word=피', '+2:word=된', '+1:word=이'],
  ['bias', '+0:word=이', '-2:word=피', '-1:word=범벅', '+2:word=김민희', '+1:word=된'],
  ['bias',
   '+0:word=된',
   '-2:word=범벅',
   '-1:word=이',
   '+2:word=표정',
   '+1:word=김민희'],
  ['bias',
   '+0:word=김민희',
   '-2:word=이',
   '-1:word=된',
   '+2:word=소름',
   '+1:word=표정'],
  ['bias',
   '+0:word=표정',
   '-2:word=된',
   '-1:word=김민희',
   '+2:word=쫙',
   '+1:word=소름'],
  ['bias', '+0:word=소름', '-2:word=김민희', '-1:word=표정', '+1:word=쫙'],
  ['bias', '+0:word=쫙

In [25]:
train_y

[['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'PERSON', 'O', 'O', 'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['O',
  'PERSON',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'PERSON',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'PERSON',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['O',
  'O

In [61]:
test_sents

NameError: name 'test_sents' is not defined

In [171]:
test_x

[['B',
  'E',
  'S',
  'T',
  '가',
  '슴',
  '세',
  '개',
  '녀',
  '가',
  '너',
  '무',
  '인',
  '상',
  '깊',
  '었',
  '다'],
 ['B',
  'E',
  'S',
  'T',
  '결',
  '국',
  '리',
  '콜',
  '사',
  '에',
  '서',
  '환',
  '상',
  '을',
  '끝',
  '내',
  '주',
  '게',
  '경',
  '험',
  '했',
  '다',
  '는',
  '얘',
  '기',
  '같',
  '네',
  '요',
  ',',
  '리',
  '메',
  '이',
  '크',
  '수',
  '준',
  '도',
  '괜',
  '찮',
  '았',
  '음',
  '.'],
 ['B',
  'E',
  'S',
  'T',
  '결',
  '국',
  '가',
  '상',
  '이',
  '였',
  '을',
  '까',
  '현',
  '실',
  '이',
  '였',
  '을',
  '까',
  '.',
  '.',
  '많',
  '은',
  '생',
  '각',
  '을',
  '하',
  '게',
  '만',
  '든',
  '영',
  '화'],
 ['B',
  'E',
  'S',
  'T',
  '원',
  '작',
  '의',
  '색',
  '안',
  '경',
  '을',
  '빼',
  '고',
  '보',
  '면',
  '참',
  '잘',
  '만',
  '든',
  '영',
  '화',
  ',',
  '원',
  '작',
  '팬',
  '들',
  '을',
  '위',
  '한',
  '오',
  '마',
  '쥬',
  '에',
  '도',
  '충',
  '실',
  '함'],
 ['B',
  'E',
  'S',
  'T',
  '볼',
  '만',
  '하',
  '네',
  '요',
  '.',
  '.',
  '.',
  '적',
  '이',
  '좀',
  '허',
  

In [48]:
trainer = pycrfsuite.Trainer()

In [49]:
for x, y in zip(train_x, train_y):
    trainer.append(x, y)

In [50]:
trainer.train('space.crfsuite')

Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 0
0....1....2....3....4....5....6....7....8....9....10
Number of features: 7840
Seconds required: 0.016

L-BFGS optimization
c1: 0.000000
c2: 1.000000
num_memories: 6
max_iterations: 2147483647
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 873.387602
Feature norm: 1.000000
Error norm: 978.607098
Active features: 7812
Line search trials: 1
Line search step: 0.000150
Seconds required for this iteration: 0.003

***** Iteration #2 *****
Loss: 746.182919
Feature norm: 1.169112
Error norm: 507.998302
Active features: 7840
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.002

***** Iteration #3 *****
Loss: 681.839797
Feature norm: 1.353039
Error norm: 186.506036
Active features: 7840
Line search trials: 1
Line search step: 1.000000
Seconds required for this ite

In [30]:
tagger = pycrfsuite.Tagger()
tagger.open('space.crfsuite')

In [37]:
sent = train_sents[2]
sent

[['BEST', 'O'],
 ['이선균', 'PERSON'],
 ['은', 'O'],
 ['대사', 'O'],
 ['가', 'O'],
 ['잘', 'O'],
 ['안', 'O'],
 ['들릴', 'O'],
 ['정도', 'O'],
 ['로', 'O'],
 ['가끔', 'O'],
 ['너무', 'O'],
 ['작게', 'O'],
 ['말', 'O'],
 ['한다', 'O'],
 ['.', 'O']]

In [38]:
print("Sentence: ", ' '.join(sent2words(sent)))

Sentence:  BEST 이선균 은 대사 가 잘 안 들릴 정도 로 가끔 너무 작게 말 한다 .


In [39]:
print("Correct:  ", ' '.join(sent2tags(sent)))

Correct:   O PERSON O O O O O O O O O O O O O O


In [40]:
print("Predicted:", ' '.join(tagger.tag(sent2features(sent))))

Predicted: O O O O O O O O O O O O O O O O


In [47]:
tagger.tag(['bias',
   '+0:word=김민희',
   '-2:word=이',
   '-1:word=된',
   '+2:word=소름',
   '+1:word=표정'])

['O', 'O', 'O', 'O', 'O', 'O']